In [139]:
import requests
import json
import time
import pandas as pd
import numpy as np
from WebCrawlerLibrary import gen_query_url, parse_response, extract_data_from_api
from datetime import datetime, timedelta

In [141]:
# Design dataframe
API_COLUMNS = ["Title", "Description", "NewsURL", "PublishedTime", "SourceName" , "SourceURL"]

api_content = pd.read_csv("../Data/NewsContent.csv", index_col=[0])

In [144]:
# Process query days pipeline
datetime_format = "%Y-%m-%d"
start_date = datetime(2020,2,26)
query_index_date = start_date 

status_code = 200
counter = 0
max_count = 100
first_time = False


while counter < max_count and status_code == 200 and query_index_date != datetime.now():
    # counter does not surpass 100 queryies per day
    # status_code is still successful
    # index date has yet to surpass today
    
    # Generate URLs
    query_index_date_next = query_index_date + timedelta(days=1)
    url = gen_query_url(query_index_date.strftime(datetime_format), query_index_date_next.strftime(datetime_format) )
    
    # Parse URLs
    response = requests.get(url)
    
    # get Status_code and content
    status_code, output_content = parse_response(response)
    
    # Parse content
    if status_code == 200 and output_content != "no returns":    
        new_api_data = extract_data_from_api(output_content)
    
    if counter == 0 and first_time == True:
        api_content = pd.DataFrame(new_api_data, columns = API_COLUMNS)
#         print(api_content)
    else :
        new_api_df = pd.DataFrame(new_api_data, columns = API_COLUMNS)
#         print(new_api_df)
        api_content = api_content.append(new_api_df, ignore_index = True).drop_duplicates()
    

    # Post Processing conditions
    query_index_date = query_index_date_next
    counter += 1
    if counter == max_count:
        print("Has Reached Max Count of " + str(counter))
    if status_code != 200:
        print("Status Code " + str(status_code))
    time.sleep(10)

https://gnews.io/api/v3/search?q=covid19|coronavirus us&token=837a1cfd8cff3793dca10c2b9478dcce&lang=en&country=us&mindate=2020-02-01&maxdate=2020-02-02&in=title


KeyError: 'articles'

In [ ]:
api_content
# api_content.to_csv("../Data/NewsContent.csv")